In [5]:
import os
os.getcwd()

'C:\\Users\\91760\\Uns'

In [23]:
apiKey='OFEGW7f8'
userid='P329268'
pin=2002
totpKey='2P56UKBERB54QOVM7PWEMXRCGU'

In [13]:
pip install -r requirements_dev.txt 

  Using cached autobahn-23.6.2.tar.gz (480 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached Automat-22.10.0-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl.metadata (2.2 kB)
  Using cached cffi-1.15.1.tar.gz (508 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [24 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\91760\AppData\Local\Temp\pip-install-bq1ho26p\cffi_f11468b8c16a48188113046a4ea727b7\setup.py", line 126, in <module>
      if sys.platform == "win32" and uses_msvc():
                                     ^^^^^^^^^^^
    File "C:\Users\91760\AppData\Local\Temp\pip-install-bq1ho26p\cffi_f11468b8c16a48188113046a4ea727b7\setup.py", line 105, in uses_msvc
      return config.try_compile('#ifndef _MSC_VER\n#error "not MSVC"\n#endif')
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "C:\Users\91760\anaconda3\Lib\site-packages\setuptools\_distutils\command\config.py", line 220, in try_compile
      self._compile(body, headers, include_dirs, lang)
    File "C:\Users\91760\

In [19]:
pip show smartapi-python

Name: smartapi-python
Version: 1.5.1Note: you may need to restart the kernel to use updated packages.

Summary: Angel Broking openApi integration
Home-page: https://github.com/angelbroking-github/smartapi-python
Author: ab-smartapi
Author-email: smartapi.sdk@gmail.com
License: 
Location: C:\Users\91760\anaconda3\Lib\site-packages
Requires: python-dateutil, requests, six
Required-by: 


In [5]:
pip install pyotp


Note: you may need to restart the kernel to use updated packages.  Downloading pyotp-2.9.0-py3-none-any.whl.metadata (9.8 kB)



In [25]:
from SmartApi import SmartConnect
import pyotp
obj=SmartConnect(api_key=apiKey)
data= obj.generateSession(userid, pin, pyotp. TOTP (totpKey).now())

refreshToken= data['data']['refreshToken'] 
res = obj.getProfile (refreshToken)
res

[I 241030 18:35:26 smartConnect:124] in pool


{'status': True,
 'message': 'SUCCESS',
 'errorcode': '',
 'data': {'clientcode': 'P329268',
  'name': 'PATHLAVATH  VINOD',
  'email': '',
  'mobileno': '',
  'exchanges': ['nse_fo',
   'nse_cm',
   'cde_fo',
   'ncx_fo',
   'bse_fo',
   'bse_cm',
   'mcx_fo'],
  'products': ['MARGIN', 'MIS', 'NRML', 'CNC', 'CO', 'BO'],
  'lastlogintime': '',
  'broker': ''}}

In [5]:
!pip install mibian




  Using cached mibian-0.1.3.zip (4.3 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for mibian: filename=mibian-0.1.3-py3-none-any.whl size=4036 sha256=29635161130e663a23035f379475af2f5e25f4a492a8fba8932b6661aac438af
  Stored in directory: c:\users\91760\appdata\local\pip\cache\wheels\b2\71\71\a9ff30d4851199bf1efabdda83809623cf181762214ae6b7c7
Successfully built mibian


In [9]:
!pip install yfinance


  Using cached multitasking-0.0.11-py3-none-any.whl.metadata (5.5 kB)
  Using cached peewee-3.17.7.tar.gz (939 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'


In [27]:
from SmartApi import SmartConnect
from SmartApi.smartWebSocketV2 import SmartWebSocketV2
import os
import urllib
import json
import pandas as pd
import numpy as np
import datetime as dt
from pyotp import TOTP
import threading
import mibian
import time
import yfinance as yf

In [29]:
instrument_url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
response = urllib.request.urlopen(instrument_url)
instrument_list = json.loads(response.read())

In [31]:
#function to extract all option contracts for a given ticker        
def option_contracts(ticker, option_type="CE"):
    option_contracts = []
    for instrument in instrument_list:
        if instrument["name"]==ticker and instrument["instrumenttype"] in ["OPTSTK","OPTIDX"] and instrument["symbol"][-2:]==option_type:
            option_contracts.append(instrument)
    return pd.DataFrame(option_contracts)


In [33]:
#get underlying price
underlying_price = obj.ltpData("NSE", "BANKNIFTY-EQ", "26009")["data"]["ltp"]

In [37]:
print(underlying_price)

51807.5


In [39]:
def option_chain_bear_spread(ticker, underlying_price, duration = 0, option_type="CE", exchange="NFO"):
    #duration = 0 means the closest expiry, 1 means the next closest and so on
    #num =5 means return 5 option contracts closest to the market
    df_opt_contracts = option_contracts(ticker)
    df_opt_contracts["time_to_expiry"] = (pd.to_datetime(df_opt_contracts["expiry"]) + dt.timedelta(0,16*3600) - dt.datetime.now()).dt.total_seconds() / dt.timedelta(days=1).total_seconds() # add 1 to get around the issue of time to expiry becoming 0 for options maturing on trading day   
    df_opt_contracts["strike"] = pd.to_numeric(df_opt_contracts["strike"])/100
    min_day_count = np.sort(df_opt_contracts["time_to_expiry"].unique())[duration]    
    temp = (df_opt_contracts[df_opt_contracts["time_to_expiry"] == min_day_count]).reset_index(drop=True)
    temp.sort_values(by=["strike"],inplace=True, ignore_index=True)
    atm_idx = abs(temp["strike"] - underlying_price).argmin()
    return temp.iloc[[atm_idx,atm_idx+2]]


In [53]:
opt_chain = option_chain_bear_spread("BANKNIFTY", underlying_price)


C:\Users\91760\AppData\Local\Temp\ipykernel_14584\874142782.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_opt_contracts["time_to_expiry"] = (pd.to_datetime(df_opt_contracts["expiry"]) + dt.timedelta(0,16*3600) - dt.datetime.now()).dt.total_seconds() / dt.timedelta(days=1).total_seconds() # add 1 to get around the issue of time to expiry becoming 0 for options maturing on trading day


In [55]:
opt_chain.head()

,token,symbol,name,expiry,strike,lotsize,instrumenttype,exch_seg,tick_size,time_to_expiry
61,43133,BANKNIFTY30OCT2451800CE,BANKNIFTY,30OCT2024,51800.0,15,OPTIDX,NFO,5.000000,-0.117583
63,43137,BANKNIFTY30OCT2452000CE,BANKNIFTY,30OCT2024,52000.0,15,OPTIDX,NFO,5.000000,-0.117583


In [57]:
opt_chain = opt_chain.sort_values(by=["time_to_expiry"], ignore_index=True)

In [59]:
opt_chain.head()

,token,symbol,name,expiry,strike,lotsize,instrumenttype,exch_seg,tick_size,time_to_expiry
0,43133,BANKNIFTY30OCT2451800CE,BANKNIFTY,30OCT2024,51800.0,15,OPTIDX,NFO,5.000000,-0.117583
1,43137,BANKNIFTY30OCT2452000CE,BANKNIFTY,30OCT2024,52000.0,15,OPTIDX,NFO,5.000000,-0.117583


In [49]:
tokens = opt_chain["token"].to_list()
symbol_dict = dict(zip(opt_chain.token, opt_chain.symbol))
option_data = {symbol_dict[i]:{} for i in tokens}

In [65]:
for symbol in opt_chain.symbol:
    option_data[symbol]["strike"] = opt_chain.loc[opt_chain.symbol == symbol, "strike"].to_list()[0]
    option_data[symbol]["type"] = opt_chain.loc[opt_chain.symbol == symbol, "instrumenttype"].to_list()[0]
    option_data[symbol]["time_to_expiry"] = opt_chain.loc[opt_chain.symbol == symbol, "time_to_expiry"].to_list()[0]

In [73]:
#streaming real time data
feed_token = obj.getfeedToken()
sws = SmartWebSocketV2(data["data"]["jwtToken"], userid,pin, feed_token)

correlation_id = "stream_1" #any string value which will help identify the specific streaming in case of concurrent streaming
action = 1 #1 subscribe, 0 unsubscribe
mode = 3 #1 for LTP, 2 for Quote and 3 for SnapQuote

token_list = [{"exchangeType": 2, "tokens": tokens}]

In [79]:
def on_data(wsapp, message):
    option_data[symbol_dict[message['token']]]["price"] = float(message["last_traded_price"])
    option_data[symbol_dict[message['token']]]["oi"] = int(message["open_interest"])
    option_data[symbol_dict[message['token']]]["volume"] = int(message["volume_trade_for_the_day"])
    option_data[symbol_dict[message['token']]]["bid"] = float(message["best_5_buy_data"][0]["price"])
    option_data[symbol_dict[message['token']]]["ask"] = float(message["best_5_sell_data"][0]["price"])
    option_data[symbol_dict[message['token']]]["mid_price"] = (float(message["best_5_buy_data"][0]["price"]) + float(message["best_5_sell_data"][0]["price"]))/2


In [81]:
def on_open(wsapp):
    print("on open")
    sws.subscribe(correlation_id, mode, token_list)


In [83]:
def on_error(wsapp, error):
    print(error)


In [85]:
# Assign the callbacks.
sws.on_open = on_open
sws.on_data = on_data
sws.on_error = on_error

In [87]:
def connection_fun():
    sws.connect()
    
con_thread = threading.Thread(target=connection_fun, daemon=True)
con_thread.start()
time.sleep(3)

[W 241030 19:14:38 smartWebSocketV2:319] Attempting to resubscribe/reconnect (Attempt 1)...


In [89]:
# calculate and update option greeks using real time market data
ir = 0.07


In [119]:
def on_data(wsapp, message):
    # Ensure that the message has the required fields
    try:
        option_data[symbol_dict[message['token']]]["price"] = float(message["last_traded_price"])
        option_data[symbol_dict[message['token']]]["oi"] = int(message["open_interest"])
        option_data[symbol_dict[message['token']]]["volume"] = int(message["volume_trade_for_the_day"])
        option_data[symbol_dict[message['token']]]["bid"] = float(message["best_5_buy_data"][0]["price"])
        option_data[symbol_dict[message['token']]]["ask"] = float(message["best_5_sell_data"][0]["price"])
        option_data[symbol_dict[message['token']]]["mid_price"] = (
            float(message["best_5_buy_data"][0]["price"]) + float(message["best_5_sell_data"][0]["price"])
        ) / 2
    except KeyError as e:
        print(f"Key error: {e} in message {message}")

# Assign the callbacks for WebSocket
sws.on_open = on_open
sws.on_data = on_data
sws.on_error = on_error


In [121]:
def connection_fun():
    sws.connect()

# Start the WebSocket connection in a thread
con_thread = threading.Thread(target=connection_fun, daemon=True)
con_thread.start()


In [123]:
def on_data(wsapp, message):
    print(message)  # Check if message is being received


In [125]:
# calculate and update option greeks using real time market data
ir = 0.07


In [129]:
for option in option_data:
    # Check if price is available before proceeding
    if "price" not in option_data[option]:
        print(f"Price data is missing for option: {option}, skipping...")
        continue  # Skip to the next option if price is missing

    up = obj.ltpData("NSE", "BANKNIFTY-EQ", "26009")["data"]["ltp"]
    mbo = mibian.BS([up, option_data[option]["strike"], ir, option_data[option]["time_to_expiry"]],
                    callPrice=option_data[option]["price"])
    option_data[option]["imp_vol"] = mbo.impliedVolatility

    mbo = mibian.BS([up, option_data[option]["strike"], ir, option_data[option]["time_to_expiry"]],
                    volatility=option_data[option]["imp_vol"])
    option_data[option]["delta"] = mbo.callDelta if option_data[option]["type"] == "CE" else mbo.putDelta
    option_data[option]["theta"] = mbo.callTheta if option_data[option]["type"] == "CE" else mbo.putTheta
    option_data[option]["vega"] = mbo.vega
    option_data[option]["gamma"] = mbo.gamma


Price data is missing for option: BANKNIFTY30OCT2451800CE, skipping...
Price data is missing for option: BANKNIFTY30OCT2452000CE, skipping...


In [131]:
print(f"Processing option: {option}, Data: {option_data[option]}")


Processing option: BANKNIFTY30OCT2452000CE, Data: {'strike': 52000.0, 'type': 'OPTIDX', 'time_to_expiry': -0.11758254975694446}


In [133]:
def on_data(wsapp, message):
    try:
        option_data[symbol_dict[message['token']]]["price"] = float(message["last_traded_price"])
        # Other fields like OI, volume, bid, ask, etc.
    except KeyError as e:
        print(f"Key error: {e} in message {message}")


In [135]:
def bollingerBand(df, period=20, multiple=2):
    df["ema"] = df["Adj Close"].ewm(span=period,min_periods=period).mean()
    df["bb_up"] = df["ema"] + multiple*df["Adj Close"].rolling(period).std(ddof=0)
    df["bb_dn"] = df["ema"] - multiple*df["Adj Close"].rolling(period).std(ddof=0)
    

In [137]:

def risk_reward(opt_data):
    strikes = opt_data.strike.to_list()
    price = opt_data.price.to_list()
    risk_to_reward = (float(price[0]) - float(price[1]))/(int(strikes[1]) - int(strikes[0]))
    return round(risk_to_reward,2)



In [139]:
def place_limit_order(order_params):
    response = obj.placeOrder(order_params)
    return response

In [141]:
def order_status_check(ord_id):
    #this function stops the code execution unless the order_id's status turns to COMPLETE
    pending_complete = True
    while pending_complete:
        orders = obj.orderBook()
        orders_df = pd.DataFrame(orders["data"])
        orderstatus = orders_df.loc[orders_df.orderid==ord_id, "orderstatus"].to_list()[0]
        if orderstatus=="complete":
            break
        time.sleep(10)

In [143]:
def placeBasketOrder(order_param_list, assure_execution=[0]):
    for count, order in enumerate(order_param_list):
        order_id = place_limit_order(order)
        if count in assure_execution:
            order_status_check(order_id)  

In [145]:
def create_order_params(opt_data):            
    order_params =  [{
                    "variety":"NORMAL",
                    "tradingsymbol": opt_chain.symbol.to_list()[1],
                    "symboltoken": opt_chain.token.to_list()[1],
                    "transactiontype": "BUY",
                    "exchange":"NFO",
                    "ordertype":"LIMIT",
                    "producttype":"CARRYFORWARD",
                    "duration":"DAY",
                    "price":opt_data.price.to_list()[1],
                    "quantity": opt_chain.lotsize.to_list()[1]
                    },
                    {
                    "variety":"NORMAL",
                    "tradingsymbol": opt_chain.symbol.to_list()[0],
                    "symboltoken": opt_chain.token.to_list()[0],
                    "transactiontype": "SELL",
                    "exchange":"NFO",
                    "ordertype":"LIMIT",
                    "producttype":"CARRYFORWARD",
                    "duration":"DAY",
                    "price":opt_data.price.to_list()[0],
                    "quantity": opt_chain.lotsize.to_list()[0]
                    }]
    
    return order_params

In [169]:
def strategy(opt_data):
    pos_df = pd.DataFrame(obj.position()["data"])
    holding_df = pd.DataFrame(obj.holding()["data"])
    ord_df = pd.DataFrame(obj.orderBook()["data"])
    
    option_data_df = pd.DataFrame(opt_data).T
    
    if len(holding_df)>0:
        if len([i for i in holding_df.tradingsymbol if i in option_data_df.index]) > 0:
            return
    
    if len(pos_df)>0:
        if len([i for i in pos_df.tradingsymbol if i in option_data_df.index]) > 0:
            return
    
    if len(ord_df)>0:
        if len([i for i in ord_df.tradingsymbol if i in option_data_df.index]) > 0:
            return
    
    ohlc = yf.download("^NSEBANK", period='55d', interval='5m')
    #ohlc = fetchOHLC(ticker,"5minute",4)
    bollingerBand(ohlc)
    
    if (ohlc["Adj Close"].iloc[-2] > ohlc["bb_up"].iloc[-2]) and \
        (ohlc["Adj Close"].iloc[-1] < ohlc["bb_up"].iloc[-1]):
            if risk_reward(option_data_df) > 0.4:
                order_params = create_order_params(option_data_df)
                print("order placed")
                #placeBasketOrder(order_params)

In [ ]:
starttime = time.time()
timeout = starttime + 60*60*5  # 5 hours timeout

while time.time() <= timeout:
    try:
        strategy(option_data)
        time.sleep(300 - ((time.time() - starttime) % 300))  # Sleep for remainder of the 5-minute interval
    except Exception as e:
        print(e)